# Análise Prática da Distribuição de Weibull

Este notebook apresenta exemplos práticos de implementação e análise da Distribuição de Weibull, fundamental para engenharia de confiabilidade e análise de tempo de vida.

## Objetivos

1. Compreender os parâmetros da distribuição Weibull
2. Implementar análises de confiabilidade
3. Realizar estimação de parâmetros
4. Validar modelos com testes estatísticos
5. Aplicar em casos práticos de engenharia

## 1. Importação de Bibliotecas

Começamos importando as bibliotecas necessárias para análise estatística, visualização e modelagem Weibull.

In [ ]:
# Bibliotecas fundamentais
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import weibull_min
from scipy.optimize import minimize

# Configuração de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

# Suprimir warnings
import warnings
warnings.filterwarnings('ignore')

# Seed para reprodutibilidade
np.random.seed(42)

print('Bibliotecas importadas com sucesso!')
print(f'NumPy versão: {np.__version__}')
print(f'Pandas versão: {pd.__version__}')

## 2. Fundamentos da Distribuição de Weibull

### 2.1 Definição Matemática

A distribuição Weibull com dois parâmetros é definida por:

**Função Densidade de Probabilidade (PDF):**
$$f(t; \beta, \eta) = \frac{\beta}{\eta}\left(\frac{t}{\eta}\right)^{\beta-1} \exp\left[-\left(\frac{t}{\eta}\right)^\beta\right]$$

**Função de Distribuição Acumulada (CDF):**
$$F(t; \beta, \eta) = 1 - \exp\left[-\left(\frac{t}{\eta}\right)^\beta\right]$$

**Parâmetros:**
- $\beta > 0$: Parâmetro de forma (shape)
- $\eta > 0$: Parâmetro de escala (scale)

### 2.2 Interpretação dos Parâmetros

In [ ]:
# Visualização do efeito do parâmetro de forma (beta)
t = np.linspace(0, 5, 1000)
eta = 2  # Escala fixa
betas = [0.5, 1.0, 1.5, 2.0, 3.0]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# PDF
for beta in betas:
    pdf = weibull_min.pdf(t, beta, scale=eta)
    axes[0].plot(t, pdf, label=f'β = {beta}', linewidth=2)

axes[0].set_title('Função Densidade de Probabilidade (PDF)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Tempo (t)', fontsize=12)
axes[0].set_ylabel('f(t)', fontsize=12)
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# CDF
for beta in betas:
    cdf = weibull_min.cdf(t, beta, scale=eta)
    axes[1].plot(t, cdf, label=f'β = {beta}', linewidth=2)

axes[1].set_title('Função de Distribuição Acumulada (CDF)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Tempo (t)', fontsize=12)
axes[1].set_ylabel('F(t)', fontsize=12)
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print('Interpretação dos valores de β:')
print('β < 1: Taxa de falha decrescente (mortalidade infantil)')
print('β = 1: Taxa de falha constante (distribuição exponencial)')
print('β > 1: Taxa de falha crescente (desgaste/envelhecimento)')

## 3. Exemplo Prático: Análise de Vida Útil de Brocas Industriais

**Contexto:** Uma empresa manufatureira deseja analisar a vida útil de brocas de carboneto utilizadas na perfuração de aço inoxidável.

**Objetivo:** Estimar quando as brocas devem ser substituídas para minimizar falhas catastróficas e custos de manutenção.

In [ ]:
# Simular dados reais de falhas de brocas
# Parâmetros verdadeiros (desconhecidos na prática)
beta_true = 2.5  # Desgaste progressivo
eta_true = 2000  # Vida característica: 2000 furos

# Gerar amostra de 80 brocas
n_samples = 80
failure_times = weibull_min.rvs(beta_true, scale=eta_true, size=n_samples)

# Adicionar censura (20% das brocas ainda funcionando ao fim do estudo)
censoring_time = 3000
censored = failure_times > censoring_time
observed_times = np.where(censored, censoring_time, failure_times)
status = (~censored).astype(int)  # 1 = falha observada, 0 = censurado

# Criar DataFrame
df = pd.DataFrame({
    'Tempo_Furos': observed_times,
    'Status': status,
    'Censored': censored
})

print('Resumo dos Dados:')
print(f'Total de brocas: {n_samples}')
print(f'Falhas observadas: {status.sum()}')
print(f'Censuradas: {censored.sum()}')
print(f'\nEstatísticas Descritivas (falhas observadas):')
print(df[df['Status'] == 1]['Tempo_Furos'].describe())

# Histograma
plt.figure(figsize=(10, 5))
plt.hist(df[df['Status'] == 1]['Tempo_Furos'], bins=20, edgecolor='black', alpha=0.7)
plt.axvline(censoring_time, color='r', linestyle='--', linewidth=2, 
            label=f'Tempo de censura: {censoring_time}')
plt.xlabel('Número de Furos até Falha', fontsize=12)
plt.ylabel('Frequência', fontsize=12)
plt.title('Distribuição dos Tempos de Falha de Brocas Industriais', 
          fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 4. Estimação de Parâmetros

### 4.1 Método da Máxima Verossimilhança (MLE)

A estimação por MLE maximiza a função de verossimilhança dada os dados observados, considerando a censura.

In [ ]:
# Função de log-verossimilhança negativa com censura
def neg_log_likelihood_weibull(params, times, status):
    beta, eta = params
    if beta <= 0 or eta <= 0:
        return np.inf
    
    # Log-verossimilhança para falhas observadas
    ll_failures = status * (np.log(beta) - beta * np.log(eta) + 
                            (beta - 1) * np.log(times) - (times / eta) ** beta)
    
    # Log-verossimilhança para censurados (função de sobrevivência)
    ll_censored = (1 - status) * (-(times / eta) ** beta)
    
    return -np.sum(ll_failures + ll_censored)

# Estimação MLE
initial_guess = [1.5, 2000]
result = minimize(neg_log_likelihood_weibull, initial_guess, 
                 args=(observed_times, status), 
                 method='Nelder-Mead')

beta_hat, eta_hat = result.x

print('='*60)
print('ESTIMAÇÃO DE PARÂMETROS - MÁXIMA VEROSSIMILHANÇA (MLE)')
print('='*60)
print(f'\nParâmetro de Forma (β) estimado: {beta_hat:.4f}')
print(f'Parâmetro de Escala (η) estimado: {eta_hat:.2f} furos')
print(f'\nValores verdadeiros (para comparação):')
print(f'β verdadeiro: {beta_true}')
print(f'η verdadeiro: {eta_true}')
print(f'\nErro relativo β: {abs(beta_hat - beta_true) / beta_true * 100:.2f}%')
print(f'Erro relativo η: {abs(eta_hat - eta_true) / eta_true * 100:.2f}%')
print('='*60)

# Interpretação
print(f'\n📊 INTERPRETAÇÃO:')
if beta_hat > 1:
    print(f'Com β = {beta_hat:.2f} > 1, a taxa de falha é CRESCENTE.')
    print('Isso indica desgaste progressivo - brocas se degradam com o uso.')
elif beta_hat < 1:
    print(f'Com β = {beta_hat:.2f} < 1, a taxa de falha é DECRESCENTE.')
    print('Isso indica falhas precoces (mortalidade infantil).')
else:
    print(f'Com β ≈ 1, a taxa de falha é CONSTANTE (exponencial).')
    print('Falhas ocorrem aleatoriamente, independente da idade.')

print(f'\nEm t = η = {eta_hat:.0f} furos, 63.2% das brocas já falharam.')

## 5. Análise de Confiabilidade

### 5.1 Função de Confiabilidade R(t)

A função de confiabilidade representa a probabilidade de uma broca **não falhar** até um determinado número de furos.

In [ ]:
# Cálculos de confiabilidade
t_analysis = np.array([500, 1000, 1500, 2000, 2500, 3000])
reliability = np.exp(-(t_analysis / eta_hat) ** beta_hat)
failure_rate = (beta_hat / eta_hat) * (t_analysis / eta_hat) ** (beta_hat - 1)

# Criar tabela de resultados
results_df = pd.DataFrame({
    'Furos': t_analysis,
    'Confiabilidade R(t)': reliability,
    'Prob. Falha F(t)': 1 - reliability,
    'Taxa de Falha λ(t)': failure_rate
})

print('\n📈 ANÁLISE DE CONFIABILIDADE')
print('='*80)
print(results_df.to_string(index=False, float_format='%.4f'))
print('='*80)

# Visualização
t_plot = np.linspace(0, 4000, 1000)
R_t = np.exp(-(t_plot / eta_hat) ** beta_hat)
F_t = 1 - R_t
lambda_t = (beta_hat / eta_hat) * (t_plot / eta_hat) ** (beta_hat - 1)

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Confiabilidade
axes[0].plot(t_plot, R_t, 'b-', linewidth=2.5)
axes[0].fill_between(t_plot, R_t, alpha=0.3)
axes[0].axhline(0.9, color='r', linestyle='--', alpha=0.6, label='R = 90%')
axes[0].set_xlabel('Número de Furos', fontsize=12)
axes[0].set_ylabel('Confiabilidade R(t)', fontsize=12)
axes[0].set_title('Função de Confiabilidade', fontsize=13, fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].legend()

# Probabilidade de Falha
axes[1].plot(t_plot, F_t, 'g-', linewidth=2.5)
axes[1].fill_between(t_plot, F_t, alpha=0.3)
axes[1].set_xlabel('Número de Furos', fontsize=12)
axes[1].set_ylabel('Probabilidade de Falha F(t)', fontsize=12)
axes[1].set_title('Função de Distribuição Acumulada', fontsize=13, fontweight='bold')
axes[1].grid(True, alpha=0.3)

# Taxa de Falha
axes[2].plot(t_plot, lambda_t, 'r-', linewidth=2.5)
axes[2].set_xlabel('Número de Furos', fontsize=12)
axes[2].set_ylabel('Taxa de Falha λ(t)', fontsize=12)
axes[2].set_title('Função Taxa de Falha (Hazard)', fontsize=13, fontweight='bold')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Encontrar tempo para R = 0.90
t_90 = eta_hat * (-np.log(0.90)) ** (1 / beta_hat)
print(f'\n💡 INSIGHT: Para manter 90% de confiabilidade, substituir brocas a cada {t_90:.0f} furos.')

## 6. Validação do Modelo

### 6.1 Teste de Kolmogorov-Smirnov

Verificamos se os dados se ajustam adequadamente à distribuição Weibull estimada.

In [ ]:
# Teste KS apenas com dados não censurados
uncensored_data = observed_times[status == 1]

# Teste de Kolmogorov-Smirnov
ks_stat, ks_pvalue = stats.kstest(uncensored_data, 
                                   lambda x: weibull_min.cdf(x, beta_hat, scale=eta_hat))

print('\n🔬 TESTE DE ADERÊNCIA - KOLMOGOROV-SMIRNOV')
print('='*60)
print(f'Estatística KS: {ks_stat:.4f}')
print(f'p-valor: {ks_pvalue:.4f}')
print('\nHipóteses:')
print('H0: Os dados seguem uma distribuição Weibull')
print('H1: Os dados NÃO seguem uma distribuição Weibull')
print(f'\nNível de significância: α = 0.05')
if ks_pvalue > 0.05:
    print('✅ Conclusão: Não rejeitamos H0. O modelo Weibull é adequado.')
else:
    print('❌ Conclusão: Rejeitamos H0. O modelo Weibull pode não ser adequado.')
print('='*60)

# Gráfico QQ-Plot
plt.figure(figsize=(10, 5))

# CDF empírica vs teórica
sorted_data = np.sort(uncensored_data)
n = len(sorted_data)
empirical_cdf = np.arange(1, n + 1) / (n + 1)
theoretical_cdf = weibull_min.cdf(sorted_data, beta_hat, scale=eta_hat)

plt.subplot(1, 2, 1)
plt.plot(sorted_data, empirical_cdf, 'bo-', label='CDF Empírica', markersize=4)
plt.plot(sorted_data, theoretical_cdf, 'r-', label='CDF Weibull Teórica', linewidth=2)
plt.xlabel('Tempo (furos)', fontsize=11)
plt.ylabel('Probabilidade Acumulada', fontsize=11)
plt.title('Comparação CDF Empírica vs Teórica', fontsize=12, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

# QQ-Plot
plt.subplot(1, 2, 2)
theoretical_quantiles = weibull_min.ppf(empirical_cdf, beta_hat, scale=eta_hat)
plt.scatter(theoretical_quantiles, sorted_data, alpha=0.6, s=40)
min_val = min(theoretical_quantiles.min(), sorted_data.min())
max_val = max(theoretical_quantiles.max(), sorted_data.max())
plt.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Linha 45°')
plt.xlabel('Quantis Teóricos (Weibull)', fontsize=11)
plt.ylabel('Quantis Observados', fontsize=11)
plt.title('QQ-Plot', fontsize=12, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Otimização de Manutenção

### 7.1 Determinação do Intervalo Ótimo de Manutenção Preventiva

**Objetivo:** Minimizar o custo total esperado considerando:
- Custo de manutenção preventiva (substituição planejada)
- Custo de falha (substituição emergencial + tempo de parada)

In [ ]:
# Custos (em unidades monetárias)
C_preventive = 50  # Custo manutenção preventiva
C_failure = 500    # Custo de falha (inclui parada de produção)

def expected_cost_per_unit_time(t_pm, beta, eta, C_p, C_f):
    """
    Custo esperado por unidade de tempo.
    t_pm: Tempo de manutenção preventiva
    """
    R_t = np.exp(-(t_pm / eta) ** beta)
    # Custo esperado
    numerator = C_p * R_t + C_f * (1 - R_t)
    denominator = t_pm
    return numerator / denominator

# Avaliar custos para diferentes intervalos
t_range = np.linspace(100, 3500, 500)
costs = [expected_cost_per_unit_time(t, beta_hat, eta_hat, C_preventive, C_failure) 
         for t in t_range]

# Encontrar ótimo
optimal_idx = np.argmin(costs)
t_optimal = t_range[optimal_idx]
cost_optimal = costs[optimal_idx]

print('\n💰 OTIMIZAÇÃO DE MANUTENÇÃO PREVENTIVA')
print('='*60)
print(f'Custo Manutenção Preventiva: ${C_preventive}')
print(f'Custo de Falha: ${C_failure}')
print(f'\nIntervalo Ótimo: {t_optimal:.0f} furos')
print(f'Custo Esperado Mínimo: ${cost_optimal:.4f} por furo')
print(f'\nConfiabilidade no intervalo ótimo: {np.exp(-(t_optimal/eta_hat)**beta_hat):.2%}')
print('='*60)

# Visualização
plt.figure(figsize=(12, 6))
plt.plot(t_range, costs, 'b-', linewidth=2.5, label='Custo Esperado')
plt.axvline(t_optimal, color='r', linestyle='--', linewidth=2, 
            label=f'Ótimo: {t_optimal:.0f} furos')
plt.scatter([t_optimal], [cost_optimal], color='r', s=200, zorder=5, 
           marker='*', edgecolors='black', linewidths=1.5)
plt.xlabel('Intervalo de Manutenção Preventiva (furos)', fontsize=12)
plt.ylabel('Custo Esperado por Furo ($)', fontsize=12)
plt.title('Otimização do Intervalo de Manutenção Preventiva', 
         fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Análise de sensibilidade
print('\n📊 ANÁLISE DE SENSIBILIDADE - Variação de Custos')
print('='*60)
cost_ratios = [5, 10, 15, 20]
for ratio in cost_ratios:
    C_f_temp = C_preventive * ratio
    costs_temp = [expected_cost_per_unit_time(t, beta_hat, eta_hat, C_preventive, C_f_temp) 
                  for t in t_range]
    t_opt_temp = t_range[np.argmin(costs_temp)]
    print(f'Razão C_falha/C_preventiva = {ratio}: Intervalo ótimo = {t_opt_temp:.0f} furos')
print('='*60)

## 8. Conclusões e Recomendações

### Principais Achados:

1. **Caracterização do Desgaste:**
   - Parâmetro β > 1 confirma desgaste progressivo
   - Taxa de falha aumenta com o uso (típico de componentes mecânicos)

2. **Confiabilidade:**
   - 63,2% das brocas falham em torno de η furos
   - Modelo Weibull adequado (teste KS aprovado)

3. **Estratégia de Manutenção:**
   - Manutenção preventiva economicamente justificada
   - Intervalo ótimo balanceia custos preventivos e falhas

### Recomendações Práticas:

✅ Implementar programa de manutenção preventiva no intervalo ótimo calculado

✅ Monitorar continuamente tempos de falha para atualizar modelo

✅ Considerar condições operacionais (material perfurado, velocidade, refrigeração)

✅ Treinar operadores para identificar sinais de degradação precoce

✅ Reavaliar custos periodicamente para ajustar política de manutenção

### Próximos Passos:

- Análise de covariáveis (temperatura, pressão, velocidade)
- Implementação de sensores IoT para monitoramento em tempo real
- Desenvolvimento de dashboard de confiabilidade
- Estudo de outros componentes críticos com metodologia similar

---

## Referências

1. **Lawless, J. F.** (2003). *Statistical Models and Methods for Lifetime Data*. Wiley.
2. **Meeker, W. Q., & Escobar, L. A.** (1998). *Statistical Methods for Reliability Data*. Wiley.
3. **Nelson, W. B.** (1982). *Applied Life Data Analysis*. Wiley.
4. **Murthy, D. N. P., et al.** (2004). *Weibull Models*. Wiley.

---

**Desenvolvido para:** Curso de Análise de Dados  
**Autor:** Prof. Luis Caparroz  
**Última Atualização:** 2025